# 🤪 Conditional WGAN-GP on CelebA Faces - Train

This notebook is an **unofficial PyTorch implementation** of the excellent [Keras example](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/04_gan/03_cgan/cgan.ipynb) for conditional GAN, originally created by David Foster as part of the companion code for the excellent book [Generative Deep Learning, 2nd Edition](https://www.oreilly.com/library/view/generative-deep-learning/9781098134174/).

_The original code is available [here](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition) and is licensed under the Apache License 2.0._
_This implementation is distributed under the Apache License 2.0. See the LICENSE file for details._

In this notebook, we'll walk through the steps required to train your own Conditional GAN on the CelebA faces dataset using PyTorch

In [ ]:
%load_ext autoreload
%autoreload 2

import os

# Get the working directory and the current notebook directory
working_dir = os.getcwd()
exp_dir = os.path.join(working_dir, "notebooks/04_gan/03_cgan/")

In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
from torchinfo import summary
from torch import optim
from torch import autograd
import torch
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot as plt
import torch.utils

import pandas as pd
import PIL

from notebooks.utils import display

import math

In [ ]:
IMAGE_SIZE = 64
CHANNELS = 3
CLASSES = 2
BATCH_SIZE = 128
Z_DIM = 32
LEARNING_RATE = 0.00005
EPOCHS = 20 * 100
CRITIC_STEPS = 3
GP_WEIGHT = 10.0
LOAD_MODEL = True
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.9
LABEL = "Blond_Hair"

## 1. Prepare the data <a name="prepare"></a>

In [ ]:
data_dir = working_dir + "/data"
dataset_dir = data_dir + "/celeba-dataset"
labels_file = dataset_dir + "/list_attr_celeba.csv"
image_dir = dataset_dir + "/img_align_celeba/img_align_celeba/"

In [ ]:
# Implement a dataset for custom structure images with labels
class ImageWLabelsDataset(torch.utils.data.Dataset):
    
    def __init__(self, images_dir, images_list, labels_list, transforms,  label_transforms):
        self.images_dir = images_dir
        self.labels = labels_list
        self.transforms = transforms
        self.img_files = images_list
        self.labels_transforms = label_transforms
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = PIL.Image.open(self.images_dir + self.img_files[idx]).convert("RGB")
        label = torch.tensor(self.labels[idx])

        if self.transforms:
            image = self.transforms(image)
        if self.labels_transforms:
            label = self.labels_transforms(label)
        
        return(image, label)

In [ ]:
# Read the labels
attributes = pd.read_csv(labels_file)
print(attributes.columns)
attributes.head()

In [ ]:
LABEL = "Blond_Hair"
IMAGE_HEAD = "image_id"
attributes[LABEL].head()

image_list = attributes[IMAGE_HEAD].to_list()
labels_list = attributes[LABEL].to_list()
labels_list = [x if x==1 else 0 for x in labels_list]

print(len(image_list), len(labels_list))
print(image_list[0], labels_list[0])

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=0.5, std=0.5)
])

def one_hot_encode(label):
    return F.one_hot(label, num_classes=CLASSES).float()

dataset_wlabels = ImageWLabelsDataset(images_dir=image_dir, images_list=image_list, 
                              labels_list=labels_list, transforms=transform, label_transforms=one_hot_encode)

train_data, _ = torch.utils.data.random_split(dataset_wlabels, [0.1, 0.9])

train_data_loader_wlabels = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

print(len(train_data))


In [ ]:
train_iter = iter(train_data_loader_wlabels)
sample_images, labels = next(train_iter)

In [ ]:
display(sample_images)
print(labels[0])

## 2. Build the GAN <a name="build"></a>

In [ ]:
class Critic(nn.Module):
    def __init__(self, channels, image_size, condition_num_classes):
        super().__init__()
        self.channels = channels
        self.image_size = image_size
        self.condition_num_classes = condition_num_classes
        self.input_channels = self.channels + self.condition_num_classes

        self.dropout = nn.Dropout(0.3)
        
        # layer group 1
        p = self._get_padding_size(input_w=self.image_size, stride=2, kernal_size=4)

        self.conv_1 = nn.Conv2d(in_channels=self.input_channels, out_channels=64, 
                                kernel_size=4, stride=2, padding=p)
        
        # layer group 2 
        p = self._get_padding_size(input_w=self.image_size/2, stride=2, kernal_size=4)

        self.conv_2 = nn.Conv2d(in_channels=64, out_channels=128, 
                               kernel_size=4, stride=2, padding=p)
        
        # layer group 3
        p = self._get_padding_size(input_w=self.image_size/4, stride=2, kernal_size=4)

        self.conv_3 = nn.Conv2d(in_channels=128, out_channels=128, 
                               kernel_size=4, stride=2, padding=p)

        # layer group 4
        p = self._get_padding_size(input_w=self.image_size/8, stride=2, kernal_size=4)
        self.conv_4 = nn.Conv2d(in_channels=128, out_channels=128, 
                               kernel_size=4, stride=2, padding=p)

        self.conv_5 = nn.Conv2d(in_channels=128, out_channels=1, 
                               kernel_size=4, stride=1, padding=0)

    @staticmethod
    def _get_padding_size(input_w, stride, kernal_size):
        p = ((input_w /2) - 1) * stride
        p = (p - input_w) + kernal_size
        p = math.ceil(p/2)

        return p
    
    def forward(self, image, condition):

        B = image.shape[0]
        num_classes = condition.shape[1]

        assert num_classes == self.condition_num_classes

        # create the condition in shape B, cond_classes, image_size, image_size
        condition = condition.view(B, self.condition_num_classes, 1, 1)
        condition = condition.expand(-1, -1, self.image_size, self.image_size)

        x = torch.cat([image, condition], dim=1)
        
        x = self.conv_1(x)
        x = F.leaky_relu(x, 0.2)

        x = self.conv_2(x)
        x = F.leaky_relu(x, 0.2)
        x = self.dropout(x)

        x = self.conv_3(x)
        x = F.leaky_relu(x, 0.2)
        x = self.dropout(x)

        x = self.conv_4(x)
        x = F.leaky_relu(x, 0.2)
        x = self.dropout(x)

        x = self.conv_5(x)

        x = x.view((B,1))

        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
critic = Critic(CHANNELS, IMAGE_SIZE, CLASSES)

print(critic.state_dict)

In [ ]:
summary(critic, ((1,CHANNELS, IMAGE_SIZE, IMAGE_SIZE), (1, CLASSES)))

In [ ]:
class Generator(nn.Module):
    def __init__(self, num_dim, channels, condition_num_classes):
        super().__init__()

        self.num_dim = num_dim
        self.channels = channels
        self.condition_num_classes = condition_num_classes
        self.input_dim = self.num_dim + self.condition_num_classes

        self.conv_trans_1 = nn.ConvTranspose2d(in_channels=self.input_dim, out_channels=128, 
                                               kernel_size=4, stride=1, padding=0, bias=False, output_padding=0)
        self.bn_1 = nn.BatchNorm2d(num_features=128, momentum=0.9)

        p = self._get_padding_size(input_w=4, stride=2, kernal_size=4)
        self.conv_trans_2 = nn.ConvTranspose2d(in_channels=128, out_channels=128, 
                                               kernel_size=4, stride=2, padding=p, output_padding=0, bias=False)
        
        self.bn_2 = nn.BatchNorm2d(num_features=128, momentum=0.9)

        p = self._get_padding_size(input_w=4*2, stride=2, kernal_size=4)
        self.conv_trans_3 = nn.ConvTranspose2d(in_channels=128, out_channels=128, 
                                               kernel_size=4, stride=2, padding=p, output_padding=0, bias=False)
        self.bn_3 = nn.BatchNorm2d(num_features=128, momentum=0.9)

        p = self._get_padding_size(input_w=4*4, stride=2, kernal_size=4) 
        self.conv_trans_4 = nn.ConvTranspose2d(in_channels=128, out_channels=64, 
                                               kernel_size=4, stride=2, padding=p, output_padding=0, bias=False)
        self.bn_4 = nn.BatchNorm2d(num_features=64, momentum=0.9)

        p = self._get_padding_size(input_w=4*8, stride=2, kernal_size=4)
        self.conv_trans_5 = nn.ConvTranspose2d(in_channels=64, out_channels=self.channels, 
                                               kernel_size=4, stride=2, padding=p, output_padding=0, bias=False)

    
    @staticmethod
    def _get_padding_size(input_w, stride, kernal_size):
        p = ((input_w - 1) * stride) / 2
        p = p - input_w
        p = p + (kernal_size / 2)
        p = p + 1/2
        return math.floor(p)
    
    def forward(self, noise, condition):
        
        B = noise.shape[0]
        condition_classes = condition.shape[1]

        assert condition_classes == self.condition_num_classes

        x = torch.cat([noise, condition], dim=1)

        x = x.view((B, self.input_dim, 1, 1))
        x = self.conv_trans_1(x)
        x = self.bn_1(x)
        x = F.leaky_relu(x, 0.2)

        x = self.conv_trans_2(x)
        x = self.bn_2(x)
        x = F.leaky_relu(x, 0.2)

        x = self.conv_trans_3(x)
        x = self.bn_3(x)
        x = F.leaky_relu(x, 0.2)

        x = self.conv_trans_4(x)
        x = self.bn_4(x)
        x = F.leaky_relu(x, 0.2)

        x = self.conv_trans_5(x)
        x = F.tanh(x)

        return (x)


In [ ]:
generator = Generator(Z_DIM, CHANNELS, CLASSES)
print(generator.state_dict)

In [ ]:
summary(generator, ((1, Z_DIM), (1, CLASSES)))

CGAN class

In [ ]:
class CGAN (nn.Module):
    def __init__(self, num_dim, channels, image_size, cond_num_classes, log_dir="./log/"):
        super().__init__()
        self.num_dim = num_dim
        self.channels = channels
        self.image_size = image_size
        self.cond_num_classes = cond_num_classes
        
        self.generator = Generator(num_dim, channels, cond_num_classes)
        self.critic = Critic(channels, image_size, cond_num_classes)

        self.writer_train = SummaryWriter(log_dir + "/train")
    
    # this function will not be used, we just implment it to be able to use
    # the torchinfo summary function
    def forward(self, noise, condition):
        x = self.generator(noise, condition)
        x = self.critic(x, condition)
        return x

    def train_step(self, train_inputs):
        # set the dicremenator and generator to training mode
        self.generator.train()
        self.critic.train()

        # generate fake images
        real_images, labels = train_inputs
        B = real_images.shape[0]

        # We train the critic more times per step
        for i in range(self.critic_steps):
            # zero the grads
            self.c_optimizer.zero_grad()
            input_noise = torch.randn((B, self.num_dim)).to(self.device)

            real_images = real_images.to(device)
            labels = labels.to(device)

            fake_images = self.generator(input_noise, labels)

            # train discremeantor
            fake_pred = self.critic(fake_images, labels)
            real_pred = self.critic(real_images, labels)

            #calculate dicremenator loss 
            c_w_loss = self.c_w_loss_fn(fake_pred=fake_pred, real_pred=real_pred)
            c_gp_loss = self.c_gp_loss_fn(critic=self.critic, real_images=real_images, fake_images=fake_images, labels=labels)
            c_total_loss = c_w_loss + (self.gp_lambda * c_gp_loss)


            # calculate gradiants
            c_total_loss.backward()
            # update waits
            self.c_optimizer.step()

        # train generator

        # zero the grads
        self.g_optimizer.zero_grad()

        fake_images = self.generator(input_noise, labels)
        # train discremeantor
        fake_pred = self.critic(fake_images, labels)

        g_w_loss = self.g_w_loss_fn(fake_pred)
        #cal gradients
        g_w_loss.backward()
        #update waits
        self.g_optimizer.step()

        loss_dict = {"c_w_loss":c_w_loss.item(), "c_gp_loss":c_gp_loss.item(), 
                     "c_total_loss": c_total_loss.item(), "g_w_loss":g_w_loss.item()}
        
        return loss_dict


    def fit(self, training_dataloader, epochs, g_optimizer, c_optimizer, 
            c_w_loss_fn, c_gp_loss, g_w_loss_fn, gp_lambda, device, 
            critic_steps=3, callbacks=None):
        
        self.g_optimizer = g_optimizer
        self.c_optimizer = c_optimizer
        self.c_w_loss_fn = c_w_loss_fn
        self.g_w_loss_fn = g_w_loss_fn
        self.c_gp_loss_fn = c_gp_loss
        self.gp_lambda = gp_lambda
        self.critic_steps = critic_steps
        self.device = device


        for i in range(1, epochs+1):

            losses = {"c_w_loss_acc": 0,
                      "c_gp_loss_acc": 0,
                      "c_total_loss_acc": 0,
                      "g_w_loss_acc": 0}

            # loop over all data in the training set
            for train_data in training_dataloader:

                # run training_step
                loss_dict = self.train_step(train_data)
                losses["c_w_loss_acc"] += loss_dict["c_w_loss"]
                losses["c_gp_loss_acc"] += loss_dict["c_gp_loss"]
                losses["c_total_loss_acc"] += loss_dict["c_total_loss"]
                losses["g_w_loss_acc"] += loss_dict["g_w_loss"]
            

            losses["c_w_loss_acc"] /= len(training_dataloader)
            losses["c_gp_loss_acc"] /= len(training_dataloader)
            losses["c_total_loss_acc"] /= len(training_dataloader)
            losses["g_w_loss_acc"] /= len(training_dataloader)
            
            # print epoch progress
            print(
                f"Epoch {i}/{epochs}: Training: c_w_loss: {losses['c_w_loss_acc'] :.4f} "
                f" c_gp_loss: {losses['c_gp_loss_acc']:.4f} "
                f" c_total_loss: {losses['c_total_loss_acc']:.4f}"
                f" g_w_loss: {losses['g_w_loss_acc']:.4f}"
            )
            # log loss to tensorboard
            self.writer_train.add_scalar("c_w_loss", losses["c_w_loss_acc"], global_step=i)
            self.writer_train.add_scalar("c_gp_loss", losses["c_gp_loss_acc"], global_step=i)
            self.writer_train.add_scalar("c_total_loss",losses["c_total_loss_acc"], global_step=i)
            self.writer_train.add_scalar("g_w_loss", losses["g_w_loss_acc"], global_step=i)
            
            # run call back functions
            if callbacks is not None:
                logs = {"device":self.device,
                        "generator":self.generator,
                        "model_state_dict": self.state_dict(),
                        "loss": losses
                }

                for callback in callbacks:
                    callback.on_epoch_end(i, logs=logs)


Lose functions

In [ ]:
def wasserstein_critic_loss(fake_pred, real_pred):
    w_loss = torch.mean(fake_pred) - torch.mean(real_pred)
    return w_loss

def wasserstein_generator_loss(fake_pred):
    w_loss = -1 * torch.mean(fake_pred)
    return w_loss

def gradient_penalty(critic, real_images, fake_images, labels):
    B = real_images.shape[0]
    alpha = torch.randn((B, 1, 1, 1)).to(real_images.device)
    interpolated = (alpha * fake_images) + ((1 - alpha) * real_images)
    interpolated.requires_grad_(True)
    pred = critic(interpolated, labels)

    # calculate the gradient of the output with respect to the input
    gradients = autograd.grad(outputs=pred, inputs=interpolated, grad_outputs=torch.ones_like(pred),
                  create_graph=True, only_inputs=True)[0]
    # flaten the gradients for each image
    gradients = gradients.view(B, -1)
    # L2 norm
    # grad_norm = torch.sqrt(torch.sum(torch.square(gradients)))
    grad_norm = gradients.norm(2, dim=1)
    gp = torch.mean((grad_norm - 1)**2)

    return gp


create the required callbacks

In [ ]:
class Callback:
    def on_epoch_end(self, epoch, logs=None):
        pass

In [ ]:
class GenerateImages(Callback):
    def __init__(self, num_images, latent_dim, save_dir="./gen_examples"):
        super().__init__()
        self.num_images = num_images
        self.latent_dim = latent_dim
        self.save_dir = save_dir
    def on_epoch_end(self, epoch, logs=None):
        device = logs["device"]
        generator = logs["generator"]

        input_noise = torch.randn((self.num_images, self.latent_dim)).to(device)
        condition_zero= torch.tensor([[1.0, 0.0]] * self.num_images).to(device)
        condition_one = torch.tensor([[0.0, 1.0]] * self.num_images).to(device)

        with torch.no_grad():
            generator.eval()
            # scale back to 0 to 255
            gen_imgs_labels_zero = generator(input_noise, condition_zero).detach() * 127.5 + 127.5
            display(gen_imgs_labels_zero, save_to=self.save_dir+f"/epoch_{epoch}_lables_0.png")

            gen_imgs_labels_one = generator(input_noise, condition_one).detach() * 127.5 + 127.5
            display(gen_imgs_labels_one, save_to=self.save_dir+f"/epoch_{epoch}_lables_1.png")
        
        return


In [ ]:
class SaveCheckpoint(Callback):
    def __init__(self, save_dir, save_every=10):
        super().__init__()
        self.save_dir = save_dir
        self.save_every = save_every
    def on_epoch_end(self, epoch, logs=None):
        
        if (epoch % self.save_every) == 0:
            checkpoint = {"epoch":epoch,
                        "model_state_dict":logs["model_state_dict"],
                        "loss":logs["loss"]
                        }
            checkpoint_file = self.save_dir + f"/checkpoint_{epoch}.pth"

            torch.save(checkpoint, checkpoint_file)

Create the cgan object and train it

In [ ]:
log_dir =  exp_dir + "/log"
os.makedirs(log_dir, exist_ok=True)

sample_dir =  exp_dir + "/sample_gen"
os.makedirs(sample_dir, exist_ok=True)

checkpoint_dir =  exp_dir + "/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
cgan = CGAN(Z_DIM, CHANNELS, IMAGE_SIZE, CLASSES, log_dir).to(device)
print(cgan.state_dict)

In [ ]:
summary(cgan, ((1, Z_DIM), (1, CLASSES)))

In [ ]:
g_loss_function = wasserstein_generator_loss
c_loss_function = wasserstein_critic_loss
c_gp_loss = gradient_penalty

g_optimizer = optim.Adam(cgan.generator.parameters(), lr=LEARNING_RATE, betas=[ADAM_BETA_1, ADAM_BETA_2])
c_optimizer = optim.Adam(cgan.critic.parameters(), lr=LEARNING_RATE, betas=[ADAM_BETA_1, ADAM_BETA_2])

In [ ]:
callbacks = [GenerateImages(10, Z_DIM, save_dir=sample_dir),
             SaveCheckpoint(save_dir=checkpoint_dir, save_every=30)]

In [ ]:
# check if we have checkpoint to load
if LOAD_MODEL:
    checkpoint_file = checkpoint_dir + "/checkpoint_600.pth"
    checkpoint = torch.load(checkpoint_file)
    cgan.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
cgan.fit(train_data_loader_wlabels, epochs=EPOCHS, g_optimizer=g_optimizer, c_optimizer=c_optimizer,
          c_w_loss_fn=c_loss_function, c_gp_loss=c_gp_loss, g_w_loss_fn=g_loss_function, device=device, callbacks=callbacks,
          gp_lambda=GP_WEIGHT, critic_steps=CRITIC_STEPS)

## 3. Generate new images <a name="decode"></a>

In [ ]:
num_samples = 10
z_sample = torch.randn(size=(num_samples, Z_DIM)).to(device)

label_0 = torch.tensor([[1., 0.]] * num_samples).to(device)
imgs = cgan.generator(z_sample, label_0)
display(imgs, cmap=None)

In [ ]:
label_1 = torch.tensor([[0., 1.]] * num_samples).to(device)
imgs = cgan.generator(z_sample, label_1)
display(imgs, cmap=None)